In [1]:
import cv2
import os
import json
import numpy as np
import ast



# 读取JSON文件
json_file_path = 'codes_to_orange/codes_to_orange/orange_demo/mv_video/0134_t0575to1995/matches.json'
with open(json_file_path, 'r') as file:
    data = json.load(file)
    # print(type(data))
i = 0

# 初始化记录结果的变量
matched_results = []

# 遍历每个帧数和视角下的预测结果
for frame, frame_data in data.items():
    id_count = {}  # 记录每个id的数量
    min_distance = float('inf')  # 初始化最低相似距离为无穷大
    best_id = None
    # print("start un cycle")
    for view, view_data in frame_data.items():
        # items = view_data
        # print(items)
        # 统计每个样本中预测最多的id和最低相似距离
        # print(type(items))
        person_id_rerank = view_data[0].split('_')[0]
        # print(person_id_rerank)
        # print(predicted_id)
        success = True if view_data[1] == "True" else False
        # print(frame, view, person_id_rerank,success)
        similarity_distance = float(view_data[2])  # 直接获取相似距离字符串值
        # print(similarity_distance)
        if person_id_rerank in id_count:
            id_count[person_id_rerank] += 1
        else:
            id_count[person_id_rerank] = 1
        
    # print(id_count)

        # if similarity_distance < min_distance:
        #     min_distance = similarity_distance
        #     best_id = person_id_rerank
        #     best_view = view
            # print(best_id)

        # 如果最低相似距离小于0.4且结果为true，则记录结果
    # if  min_distance < 0.4 and best_id is not None:
    #             matched_results.append({
    #                 "frame": frame,
    #                 "view": best_view,
    #                 "best_id": best_id,
    #                 "min_distance": min_distance    
    
    # 判断是否存在众数
    max_count = max(id_count.values())
    mode_ids = [key for key, value in id_count.items() if value == max_count]
    # print(mode_ids)
    if len(mode_ids) == 1:
        best_id = mode_ids[0]
        # print(frame, min_distance, best_id, success)
        success = any([view_data[1] == "True" for view, view_data in frame_data.items() if view_data[0].split('_')[0] == best_id])
        min_distance = min([float(view_data[2]) for view, view_data in frame_data.items() if view_data[0].split('_')[0] == best_id])
        best_view = view
        # print(frame, min_distance, best_id, success, "存在众数")
    else:
        for view, view_data in frame_data.items():
            # items = view_data
            similarity_distance = float(view_data[2])
            person_id_rerank = view_data[0].split('_')[0]
            if similarity_distance < min_distance:
                min_distance = similarity_distance
                best_view = view
                best_id = person_id_rerank
                success = [view_data[1] == "True" for view, view_data in frame_data.items() if view_data[0].split('_')[0] == best_id]
                # print(frame, min_distance,best_id, success, "不存在众数")
    # 如果最低相似距离小于 0.8 且结果为 True，则记录结果
    if min_distance < 0.6 and success:
        matched_results.append({
            "frame": frame,
            "view": best_view,
            "best_id": best_id,
            "min_distance": min_distance
        })            

# 打印满足条件的结果
# print(matched_results[1])  
best_ids = [result['best_id'] for result in matched_results]
print(best_ids)             
for result in matched_results:
    print(result)
print("符合条件的结果数量为: ", len(matched_results))

['0120', '0120', '0120', '0120']
{'frame': '0700', 'view': '6', 'best_id': '0120', 'min_distance': 2.3841858e-07}
{'frame': '0705', 'view': '6', 'best_id': '0120', 'min_distance': 0.43534243}
{'frame': '0715', 'view': '6', 'best_id': '0120', 'min_distance': 0.53517765}
{'frame': '0720', 'view': '6', 'best_id': '0120', 'min_distance': 0.2483663}
符合条件的结果数量为:  4


In [2]:

import json
# 指定JSON文件路径
json_file_path = '/data02/zhangrunxiang/reid1/fast-reid/scripts/r2o.json'

# 加载JSON文件中的数据
with open(json_file_path, 'r') as file:
    json_data = json.load(file)
    
# person_id_rerank = best_ids[0]
# print(person_id_rerank)
# person_id = json_data.get(person_id_rerank)#这里就把rerank的id变成了原始id
# # print(person_id)
person_ids = []
for id in best_ids:
    if id not in person_ids:
        person_ids.append(json_data.get(id))
        
frame_numbers = []
for item in matched_results:
    frame_number = item['frame']  # 获取当前字典中的帧数信息
    frame_numbers.append(frame_number)  # 将帧数信息添加到列表中

# print(frame_numbers)


# 指定JSON文件目录路径
json_directory = '/data02/zhangrunxiang/reid1/datasets/Wildtrack_dataset_full/Wildtrack_dataset/annotations_positions/'

# 遍历matched_results，提取所有的帧数信息
# frame_numbers_to_search = set(item[1] for item in matched_results)
desired_position = {}  # 用于存储行人ID、帧数和视角为5的信息

for frame_number in frame_numbers:
    # 构建要查找的JSON文件名
    frame_number = str(frame_number)
    json_file_name = '0000' + frame_number + '.json'
    json_file_path = json_directory + json_file_name

    # 加载JSON文件中的数据
    with open(json_file_path, 'r') as file:
        json_data = json.load(file)
        desired_info = None
        # person_id = int(person_id)
        # for person_info in json_data:
        #     if person_info.get("personID") == person_id:
        #         for view_info in person_info.get("views", []):
        #             if view_info.get("viewNum") == 5:  # 视角为5对应c6
        #                 desired_position[(frame_number)] = view_info
        #                 break
        for person_id in person_ids:
            person_id = int(person_id)
            for person_info in json_data:
                if person_info.get("personID") == person_id:
                    for view_info in person_info.get("views", []):
                        if view_info.get("viewNum") == 5:
                            desired_position[(frame_number)] = view_info
                            break
print("desired_position: ", desired_position)



desired_position:  {'0700': {'viewNum': 5, 'xmax': 213, 'xmin': 184, 'ymax': 224, 'ymin': 112}, '0705': {'viewNum': 5, 'xmax': 183, 'xmin': 154, 'ymax': 225, 'ymin': 113}, '0715': {'viewNum': 5, 'xmax': 123, 'xmin': 91, 'ymax': 230, 'ymin': 113}, '0720': {'viewNum': 5, 'xmax': 112, 'xmin': 80, 'ymax': 233, 'ymin': 113}}


In [3]:
import cv2
import os
import shutil

# 指定要处理的图像文件路径和保存路径
file_path = '/data02/zhangrunxiang/reid1/datasets/Wildtrack_dataset_full/wildtrack_dataset/Image_subsets/C6/'
save_path = '/data02/zhangrunxiang/reid1/datasets/wildtrack_dataset_full/Wildtrack_dataset/Image_subsets/C6/'

# 创建保存路径文件夹
os.makedirs(save_path, exist_ok=True)

start_frame = 700
end_frame = 900
step = 5

# distinct_colors = [
#     (255, 0, 0),      # 蓝色
#     (0, 255, 0),      # 绿色
#     (0, 0, 255),      # 红色
#     (255, 255, 0),    # 青色
#     (255, 0, 255),    # 紫色
#     (0, 255, 255),    # 黄色
#     (128, 0, 0),      # 深红色
#     (0, 128, 0),      # 深绿色
#     (0, 0, 128),      # 深蓝色
#     (128, 128, 0),    # 橄榄色
#     (128, 0, 128),    # 紫红色
# ]


# 复制文件到目标路径,视角下对应帧数图片复制到目标路径
# for video_frame in range(start_frame, end_frame+1, step):
#     source_file = os.path.join(file_path, f'{video_frame:08d}.png')  # 根据序号生成源文件路径
#     destination_file = os.path.join(save_path, f'{video_frame:08d}.png')  # 根据序号生成目标文件路径
#     shutil.copy(source_file, destination_file)
# print("所有图片已成功复制到路径:", save_path)

# 遍历desired_position中所有帧数信息
for frame_number, info in desired_position.items():
    frame_number = str(frame_number)
    # 构建图像文件名
    image_filename = '0000' + frame_number + '.png'
    image_path = save_path + image_filename

    # 加载图像
    image = cv2.imread(image_path)

    # 获取坐标信息
    xmin = info['xmin']
    ymin = info['ymin']
    xmax = info['xmax']
    ymax = info['ymax']

    # 在图像上绘制矩形框
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

    # 构建保存路径和文件名
    save_filename =  image_filename
    save_image_path = os.path.join(save_path, save_filename)

    # 保存带有框的图像
    cv2.imwrite(save_image_path, image, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])


# print("带有框的图像已保存在路径:", save_path)
